In [1]:
import pandas as pd
import json
from lingua import Language, LanguageDetectorBuilder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from selenium import webdriver
from selenium.webdriver.edge.options import Options

In [2]:
path = r'C:\Users\leoso\Projects\Amazon Scraping\Livros Amazon.xlsx'

df_feminism = pd.read_excel(path, sheet_name='Feminism_DB')
df_feminism_physical_media = pd.read_excel(path, sheet_name='Feminism_Physical_Media')
df_feminism_kindle_only = pd.read_excel(path, sheet_name='Feminism_Kindle_Only')
df_antifeminism = pd.read_excel(path, sheet_name='AntiFeminism_DB')
df_antifeminism_physical_media = pd.read_excel(path, sheet_name='AntiFeminism_Physical_Media')
df_antifeminism_kindle_only = pd.read_excel(path, sheet_name='AntiFeminism_Kindle_Only')

with open(r'C:\Users\leoso\Projects\Amazon Scraping\feminism_media_type_correction.json', encoding='utf-8') as json_file:
    feminism_media_type_correction = json.load(json_file)
with open(r'C:\Users\leoso\Projects\Amazon Scraping\antifeminism_media_type_correction.json', encoding='utf-8') as json_file:
    antifeminism_media_type_correction = json.load(json_file)

In [4]:
df_feminism_media_type_correction = pd.DataFrame.from_dict(
    feminism_media_type_correction,
    orient='index'
)

df_feminism_media_type_correction.index.name = 'Link'
df_feminism_media_type_correction = df_feminism_media_type_correction.reset_index()

cols = ['Link'] + [c for c in df_feminism_media_type_correction.columns if c != 'Link']
df_feminism_media_type_correction = df_feminism_media_type_correction[cols]

df_antifeminism_media_type_correction = pd.DataFrame.from_dict(
    antifeminism_media_type_correction,
    orient='index'
)

df_antifeminism_media_type_correction.index.name = 'Link'
df_antifeminism_media_type_correction = df_antifeminism_media_type_correction.reset_index()

cols = ['Link'] + [c for c in df_antifeminism_media_type_correction.columns if c != 'Link']
df_antifeminism_media_type_correction = df_antifeminism_media_type_correction[cols]

In [5]:
columns_order = [
    'Nome', 'Link', 'Editora', 'Data', 'Edição', 'Autor 1', 'Autor 2',
    'Ranking Feminismo', 'Ranking Livros', 'Categoria 1', 'Ranking Categoria 1',
    'Categoria 2', 'Ranking Categoria 2', 'Categoria 3', 'Ranking Categoria 3',
    'Tipo Livro 1', 'Preço 1', 'Tipo Livro 2', 'Preço 2',
    'Tipo Livro 3', 'Preço 3', 'Tipo Livro 4', 'Preço 4',
    'ASIN', 'ISBN-10', 'ISBN-13'
]

def merge_and_clean(media_df, original_df):
    return (
        pd.merge(media_df, original_df, how='left', on='Link')
        .drop(['Tipo Livro 1_y', 'Preço 1_y', 'Tipo Livro 2_y', 'Preço 2_y'], axis=1)
        .rename(columns={
            'Tipo Livro 1_x': 'Tipo Livro 1', 'Preço 1_x': 'Preço 1',
            'Tipo Livro 2_x': 'Tipo Livro 2', 'Preço 2_x': 'Preço 2'
        })[columns_order]
    )

df_feminism_filtered = merge_and_clean(df_feminism_media_type_correction, df_feminism)
df_antifeminism_filtered = merge_and_clean(df_antifeminism_media_type_correction, df_antifeminism)

In [6]:
df_feminism_physical_media = pd.concat([df_feminism_physical_media, df_feminism_filtered], ignore_index=True)
df_antifeminism_physical_media = pd.concat([df_antifeminism_physical_media, df_antifeminism_filtered], ignore_index=True)

def filter_kindle_only(df_kindle, df_ref):
    """
    Mantém em df_kindle apenas as linhas em que
    Link NÃO está em df_ref.Link
    OU Nome NÃO está em df_ref.Nome.
    """
    mask = (
        ~df_kindle['Link'].isin(df_ref['Link']) |
        ~df_kindle['Nome'].isin(df_ref['Nome'])
    )
    return df_kindle[mask]

df_feminism_kindle_only = filter_kindle_only(df_feminism_kindle_only, df_feminism_filtered)
df_antifeminism_kindle_only = filter_kindle_only(df_antifeminism_kindle_only, df_antifeminism_filtered)

In [7]:
dfs_dict = {
    'df_feminism_physical_media': df_feminism_physical_media,
    'df_feminism_kindle_only': df_feminism_kindle_only,
    'df_feminism_media_type_correction': df_feminism_media_type_correction,
    'df_feminism_filtered': df_feminism_filtered,
    'df_antifeminism_physical_media': df_antifeminism_physical_media,
    'df_antifeminism_kindle_only': df_antifeminism_kindle_only,
    'df_antifeminism_media_type_correction': df_antifeminism_media_type_correction,
    'df_antifeminism_filtered': df_antifeminism_filtered
}

for name, df in dfs_dict.items():
    print(f'{name}:\n- {len(df)} linhas\n- Colunas: {list(df.columns)}')

'''
Os DFs df_feminism_physical_media, df_antifeminism_physical_media, df_feminism_kindle_only e df_antifeminism_kindle_only irão substituir suas respectivas versões no arquivo Excel
'''

df_feminism_physical_media:
- 1112 linhas
- Colunas: ['Nome', 'Link', 'Editora', 'Data', 'Edição', 'Autor 1', 'Autor 2', 'Ranking Feminismo', 'Ranking Livros', 'Categoria 1', 'Ranking Categoria 1', 'Categoria 2', 'Ranking Categoria 2', 'Categoria 3', 'Ranking Categoria 3', 'Tipo Livro 1', 'Preço 1', 'Tipo Livro 2', 'Preço 2', 'ASIN', 'ISBN-10', 'ISBN-13', 'Tipo Livro 3', 'Preço 3', 'Tipo Livro 4', 'Preço 4']
df_feminism_kindle_only:
- 75 linhas
- Colunas: ['Nome', 'Link', 'Editora', 'Data', 'Edição', 'Autor 1', 'Autor 2', 'Ranking Feminismo', 'Ranking Livros', 'Categoria 1', 'Ranking Categoria 1', 'Categoria 2', 'Ranking Categoria 2', 'Categoria 3', 'Ranking Categoria 3', 'Tipo Livro 1', 'Preço 1', 'Tipo Livro 2', 'Preço 2', 'ASIN', 'ISBN-10', 'ISBN-13']
df_feminism_media_type_correction:
- 125 linhas
- Colunas: ['Link', 'Tipo Livro 1', 'Preço 1', 'Tipo Livro 2', 'Preço 2', 'Tipo Livro 3', 'Preço 3', 'Tipo Livro 4', 'Preço 4']
df_feminism_filtered:
- 125 linhas
- Colunas: ['Nome', 'Lin

'\nOs DFs df_feminism_physical_media, df_antifeminism_physical_media, df_feminism_kindle_only e df_antifeminism_kindle_only irão substituir suas respectivas versões no arquivo Excel\n'

In [8]:
detector = LanguageDetectorBuilder.from_languages(
    Language.PORTUGUESE,
    Language.ENGLISH,
    Language.SPANISH
).build()

def detect_language(text: str) -> str:
    lang = detector.detect_language_of(text)
    return lang.iso_code_639_1.name

def detect_confidences(text: str) -> dict:
    cvs = detector.compute_language_confidence_values(text)
    result = {'conf_pt': 0.0, 'conf_en': 0.0, 'conf_es': 0.0}
    for cv in cvs:
        code = cv.language.iso_code_639_1.name.lower()
        key = f'conf_{code}'
        result[key] = cv.value  
    return result


In [9]:
df_feminism_physical_media_final = df_feminism_physical_media.copy()
df_feminism_physical_media_final['lingua_predita'] = df_feminism_physical_media_final['Nome'].apply(detect_language)
conf_df = df_feminism_physical_media_final['Nome'] \
    .apply(detect_confidences) \
    .apply(pd.Series)
df_feminism_physical_media_final = pd.concat([df_feminism_physical_media_final, conf_df], axis=1)
df_feminism_physical_media_final = df_feminism_physical_media_final.drop([
    172,379,416,430,460,470,545,605,623,363,641,652,669,702,705,
    721,731,745,763,768,770,795,833,879,882,897,907,908,931,950,
    958,968,970,978,979,1004,1054,1056,1063,1071,1072,1073,1074,
    1083,1084,1086,1089,1090,1091,1092,1093,1097,1100,1107,1108,
    1111
])

df_feminism_kindle_only_final = df_feminism_kindle_only.copy()
df_feminism_kindle_only_final['lingua_predita'] = df_feminism_kindle_only_final['Nome'].apply(detect_language)
conf_df = df_feminism_kindle_only_final['Nome'] \
    .apply(detect_confidences) \
    .apply(pd.Series)
df_feminism_kindle_only_final = pd.concat([df_feminism_kindle_only_final, conf_df], axis=1)

df_feminism_kindle_only_final = df_feminism_kindle_only_final.drop([100,103,156,168,192])

df_antifeminism_physical_media_final = df_antifeminism_physical_media.copy()
df_antifeminism_physical_media_final['lingua_predita'] = df_antifeminism_physical_media_final['Nome'].apply(detect_language)
conf_df = df_antifeminism_physical_media_final['Nome'] \
    .apply(detect_confidences) \
    .apply(pd.Series)
df_antifeminism_final = pd.concat([df_antifeminism_physical_media_final, conf_df], axis=1)
df_antifeminism_final[df_antifeminism_final['lingua_predita'] != 'PT']
df_antifeminism_physical_media_final = df_antifeminism_physical_media_final.drop([21,38,40,42,43,44,45,46,47,48,49,51,54,56,58,59,60,61,62,63,64])

df_antifeminism_kindle_only_final = df_antifeminism_kindle_only.copy()
df_antifeminism_kindle_only_final['lingua_predita'] = df_antifeminism_kindle_only_final['Nome'].apply(detect_language)
conf_df = df_antifeminism_kindle_only_final['Nome'] \
    .apply(detect_confidences) \
    .apply(pd.Series)
df_antifeminism_kindle_only_final = pd.concat([df_antifeminism_kindle_only_final, conf_df], axis=1)

df_antifeminism_kindle_only_final = df_antifeminism_kindle_only_final.drop(11)

In [ ]:
path = r'C:\Users\leoso\Projects\Amazon Scraping\Livros Amazon - Copia.xlsx'

sheets_dfs_dict = {
    'Feminism_Physical_Media': df_feminism_physical_media_final,
    'Feminism_Kindle_Only': df_feminism_kindle_only_final,
    'AntiFeminism_Physical_Media': df_antifeminism_physical_media_final,
    'AntiFeminism_Kindle_Only': df_antifeminism_kindle_only_final
}

for sheet, df in sheets_dfs_dict.items():
    with pd.ExcelWriter(path,
                        engine='openpyxl',
                        mode='a',                   # abre em modo append
                        if_sheet_exists='replace'   # substitui a aba existente
                    ) as writer:
        df.to_excel(writer,
                    sheet_name=sheet,
                    index=False)

In [ ]:
def init_driver():
    options = Options()
    # Connect to an already running Edge debugger session if needed.
    options.debugger_address = "localhost:8989"
    # Add window size before instantiating the driver.
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Edge(options=options)
    return driver

driver = init_driver()

for link in df_feminism_kindle_only_final['Link']:
    driver.current_window_handle
    # abre uma aba em branco
    driver.execute_script("window.open('');")
    # muda o foco para a aba recém-criada
    driver.switch_to.window(driver.window_handles[-1])
    # navega até o link
    driver.get(link)